In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, BatchNormalization, Flatten, Reshape, LeakyReLU, Activation
from keras.activations import sigmoid

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets('./', one_hot=True)
image_size = (28, 28, 1)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [3]:
discriminator = Sequential()
discriminator.add(Conv2D(32, (5, 5), strides=2, activation=LeakyReLU(),
                 padding='same', input_shape=image_size))
discriminator.add(Conv2D(64, (5, 5), strides=2,
                 padding='same', input_shape=image_size))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU())
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3137      
Total params: 55,489.0
Trainable params: 55,361.0
Non-trainable params: 128.0
________________________________________________________________

In [4]:
z_dim = 100
depth = 128
generator = Sequential()
generator.add(Dense(7*7*depth, input_dim=z_dim))
generator.add(Reshape((7,7,depth)))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

depth = int(depth/2)
generator.add(Conv2DTranspose(depth, 5, padding='same', strides=2))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

depth = int(depth/2)
generator.add(Conv2DTranspose(depth, 5, padding='same', strides=1))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

depth = int(depth/2)
generator.add(Conv2DTranspose(depth, 5, padding='same', strides=1))
generator.add(BatchNormalization())
generator.add(LeakyReLU())

# depth = int(depth/2)
generator.add(Conv2DTranspose(1, 5, padding='same', strides=2))
generator.add(BatchNormalization())
generator.add(Activation(sigmoid))

generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 14, 14, 64)        0         
__________

In [5]:
adversarial_model = Sequential()
adversarial_model.add(generator)
adversarial_model.add(discriminator)
adversarial_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
adversarial_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 28, 28, 1)         903749    
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 55489     
Total params: 959,238.0
Trainable params: 958,628.0
Non-trainable params: 610.0
_________________________________________________________________


In [6]:
def mnist_gen(batch_size):
    while 1:
        data = mnist.train.next_batch(batch_size)
        yield data[0].reshape((-1,)+image_size)

In [7]:
batch_size = 128
mnist_data = mnist_gen(batch_size)
# for i in range(1000):
z = np.random.randn(batch_size, z_dim)
gen_im = generator.predict(z)
mnist_batch = next(mnist_data)
x = np.vstack([mnist_batch, gen_im])
y = np.ones((2*batch_size, 1), dtype='float32')
y[batch_size:] = 0
discriminator.train_on_batch(x, y)

y = np.ones((batch_size, 1))
adversarial_model.train_on_batch(z, y)

InvalidArgumentError: You must feed a value for placeholder tensor 'dense_2_input' with dtype float
	 [[Node: dense_2_input = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_2_input', defined at:
  File "/root/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/root/miniconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  Fi

In [9]:
discriminator.train_on_batch(gen_im, np.ones((128, 1)))

InvalidArgumentError: You must feed a value for placeholder tensor 'dense_2_input' with dtype float
	 [[Node: dense_2_input = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_2_input', defined at:
  File "/root/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/root/miniconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  Fi

In [ ]:
discriminator.summary()

In [ ]:
x.dtype

In [ ]:
y.dtype

In [ ]:
y = np.ones((batch_size, 1))
adversarial_model.train_on_batch(z, y)

In [ ]:
z.shape

In [ ]:
x.shape

In [ ]:
x.shape

In [ ]:
gen_im

In [ ]:
plt.imshow(gen_im[0][:,:,0])

In [ ]:
next(mnist_data)

In [ ]:
gen_im.shape

In [ ]:
gen_im.shape